# Sequence Generation

in this notebook we generate novel sequences of text based on a corpus of training data consisting of Shakespeare's complete works.

### Import libraries

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

### Download Data

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


### Import Data

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# Take a Look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The Unique Characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


### Convert Characters to Numerical Index Representation

In [12]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [13]:
print('{')
for char,_ in zip(char2idx, range(20)):
  print('  {:4s} : {:3d},'.format(repr(char), char2idx[char]))
print('   ....\n')


{
  '\n' :   0,
  ' '  :   1,
  '!'  :   2,
  '$'  :   3,
  '&'  :   4,
  "'"  :   5,
  ','  :   6,
  '-'  :   7,
  '.'  :   8,
  '3'  :   9,
  ':'  :  10,
  ';'  :  11,
  '?'  :  12,
  'A'  :  13,
  'B'  :  14,
  'C'  :  15,
  'D'  :  16,
  'E'  :  17,
  'F'  :  18,
  'G'  :  19,
   ....



### Create Training Examples And Targets

In [14]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create stream of characters for training dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [15]:
# Each sequenc has 101 characters ; we'll predict next char given the preceding ones, up to 100 chars of input 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [16]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [18]:
for input_example , traget_example in dataset.take(1):
  print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Input data: ', repr(''.join(idx2char[traget_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Input data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [19]:

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], traget_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s}".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i'
Step    1
  input: 47 ('i')
  expected output: 56 ('r'
Step    2
  input: 56 ('r')
  expected output: 57 ('s'
Step    3
  input: 57 ('s')
  expected output: 58 ('t'
Step    4
  input: 58 ('t')
  expected output: 1 (' '


### Shuffle Data For Stochastic Gradient Descent

In [20]:
# Batch size
BATCH_SIZE = 64


# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory . Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Design Neural Network Architecture

In [21]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units,
                          return_sequences=True,
                          stateful=True, # Last state for index i in batch is used as initial state
                          recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)                    
  ])
  return model

In [23]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [24]:
# Examine the shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
sampled_indices

array([33, 17, 60, 26, 26,  8, 34, 21, 16, 58, 11, 53, 30, 45, 15, 61, 27,
       22,  9, 15, 47, 62, 61, 24,  3, 22, 63, 56, 30, 41, 56, 64, 50, 23,
       41, 17, 38, 59, 50, 62, 51,  4, 37,  3, 47, 11, 40, 58, 22, 34, 39,
       56, 56,  9, 37,  8, 60, 26, 42, 30, 62, 11,  0, 36, 56, 35, 44, 48,
       34, 18, 29, 14, 55, 22, 38, 30, 60,  6, 17,  5,  0, 53, 15, 61,  5,
        5,  9, 58, 21, 21, 54, 57, 47, 64, 52, 27, 64, 62,  4,  4])

In [28]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "rs do wash the blood away.\nKeep thou the napkin, and go boast of this:\nAnd if thou tell'st the heavy"

Next Char Predictions: 
 "UEvNN.VIDt;oRgCwOJ3CixwL$JyrRcrzlKcEZulxm&Y$i;btJVarr3Y.vNdRx;\nXrWfjVFQBqJZRv,E'\noCw''3tIIpsiznOzx&&"


### Compile Model

In [29]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174514


In [30]:
model.compile(optimizer='adam', loss=loss)

In [31]:
# Directory where the checkpoints will be saved
checkpoint_dir = './model_output/seqGen'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Train !

In [32]:
EPOCHS=30

In [33]:
history= model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 12s 54ms/step - loss: 2.6717
Epoch 2/30
172/172 [==============================] - 10s 54ms/step - loss: 1.9718
Epoch 3/30
172/172 [==============================] - 11s 55ms/step - loss: 1.7014
Epoch 4/30
172/172 [==============================] - 11s 56ms/step - loss: 1.5495
Epoch 5/30
172/172 [==============================] - 11s 58ms/step - loss: 1.4599
Epoch 6/30
172/172 [==============================] - 12s 56ms/step - loss: 1.3992
Epoch 7/30
172/172 [==============================] - 11s 57ms/step - loss: 1.3531
Epoch 8/30
172/172 [==============================] - 11s 58ms/step - loss: 1.3144
Epoch 9/30
172/172 [==============================] - 11s 57ms/step - loss: 1.2800
Epoch 10/30
172/172 [==============================] - 11s 56ms/step - loss: 1.2472
Epoch 11/30
172/172 [==============================] - 11s 57ms/step - loss: 1.2148
Epoch 12/30
172/172 [==============================] - 11s 57ms/step - loss: 1.1836
E

### Rebuild model with single output for generating text one char at a Time

In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

'./model_output/seqGen/ckpt_30'

In [35]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [37]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the char returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted char as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [38]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: but me trumpet?

Lord MaKus, men lions hard!

LADY CAPULET:
O me, the bloody stard and fear
Lumbering honour on 's is the vouch.
Come, my desires; lume! 'tis enough this well-proceedings here.
Come, I, rotuen!
And yet I bear a Montague, pray?
Come, your father chances have you do but his best mars beguild now:
Methinks thou didst brak it, some; but that
Mine honour meanly hath left again
I will not be amended; but that you joy were my hell.
Abear usurp'd: pray you, away.

ANGELO:
What, art songled.

DUKE VINCENTIO:
Have not the cause?

MONTAGUE:
But never took the trranmen are in arrival,
His tempter or more benefit
which I but weak bring to be wit, some other.
But when for sweet is this woman.

DUKE OF AUMERLE:
Unto miser eyes liege.

Both:
Well, being on him?

NORTHUMBERLAND:
Then further, divers unexperl'd; and then have wounted 'go
My prius applared my mother cowague,
And here it is.
I did betider,
And to misut was the Duke of Hereford?

LEONTES:
Well, Come:
Hast the offer s

In [39]:
print(generate_text(model, start_string=u"HAMLET: To be, or not to be: that is the question:"))

HAMLET: To be, or not to be: that is the question: I will live,
That, all; as you can swift again
Where every weak sanctued and sayisfel,
As may appear what I will hear me.
If I put forth to credit bloods and that word's idle:
I give a state all things in: and let the clouds,
as beggar's sake, but stops he open arry;
Which, puts it much distraved born.

CLYORK:
I said so haste; 'tis nursed: so you are?

PAULINA:
So mumble me, Isabel, trust me, desperate, it straight:
Tell us, though rest above him hence in haze
The shapen rotten fews I bold with;
For me to cross this well-pestir, neither will I strive I give him sacreft.

FLORIZEL:
Why holingland's dagger on ren
They did request in question am I the duke.

GLOUCESTER:
Then let's awake him hence in have seems,
Or brother witchcrofth is proug withterous;
well the tribunes of the justice,
How he might I,
Thou hast forced to seem them that did usurd
And prove confession, doubting hence,
And your woss banish'd; for our own bear heart
That a